# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 150000 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "SpaceShooter" # Name of the training environment file.
curriculum_file = "Curriculum.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 32 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'SpaceShooterAcademy' started successfully!


Unity Academy name: SpaceShooterAcademy
        Number of brains: 2
        Reset Parameters :
		hazards -> 3.0
		hazards_count -> 3.0
Unity brain name: SpaceNavigatorBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 8
        Action space type: discrete
        Action space size (per agent): 8
        Memory space size (per agent): 0
        Action descriptions: None, Up, Down, Left, Right, Fire at asteroid, Fire at alien ship, Fire at alien missile
Unity brain name: SpaceShooterBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 8
        Action space type: discrete
        Action space size (per agent): 3
        Memory space size (per agent): 0
        Action descriptions: Fire at asteroid, Fire at alien ship, Fire at alien missile


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 1000. Mean Reward: 3.3178841121320746. Std of Reward: 1.5464012694796956.


INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	hazards -> 2.0, hazards_count -> 3.0


Step: 2000. Mean Reward: 2.2536873410788765. Std of Reward: 1.372060356839031.
Step: 3000. Mean Reward: 1.8222275349519084. Std of Reward: 1.888033940677971.
Step: 4000. Mean Reward: 1.5715552846512. Std of Reward: 1.4101016654776415.
Step: 5000. Mean Reward: 1.7874030037060595. Std of Reward: 1.8541378937066426.
Step: 6000. Mean Reward: 1.5890502656440708. Std of Reward: 2.107685153808033.
Step: 7000. Mean Reward: 2.3988473798367824. Std of Reward: 2.4006105261440527.
Step: 8000. Mean Reward: 2.6602720273834803. Std of Reward: 2.1591999427358535.


INFO:unityagents:
Lesson changed. Now in Lesson 2 : 	hazards -> 3.0, hazards_count -> 4.0


Step: 9000. Mean Reward: 3.6585142642127755. Std of Reward: 2.7667932438395346.
Step: 10000. Mean Reward: 4.109208797199947. Std of Reward: 2.641480150420117.
Step: 11000. Mean Reward: 3.4894162918074003. Std of Reward: 2.5395295986404016.
Step: 12000. Mean Reward: 3.5514105708236796. Std of Reward: 2.673805874229244.
Step: 13000. Mean Reward: 2.8792380291711708. Std of Reward: 2.6168118264824494.
Step: 14000. Mean Reward: 3.399885445213277. Std of Reward: 3.1031291393419522.
Step: 15000. Mean Reward: 3.616912027532659. Std of Reward: 3.5780362358389186.


...

Step: 65000. Mean Reward: 6.371633719592058. Std of Reward: 5.424866467603259.
Step: 66000. Mean Reward: 5.728848874291099. Std of Reward: 5.293688063441778.
Step: 67000. Mean Reward: 5.923108409560415. Std of Reward: 5.151138575204245.
Step: 68000. Mean Reward: 5.425693870981734. Std of Reward: 5.070574616469505.
Step: 69000. Mean Reward: 5.167465371422355. Std of Reward: 5.004238542241832.
Step: 70000. Mean Reward: 4.28244033981406. Std of Reward: 4.963513394863455.
Step: 71000. Mean Reward: 4.63915489896784. Std of Reward: 4.988766732161587.
Step: 72000. Mean Reward: 4.955716339747201. Std of Reward: 5.232641222114726.
Step: 73000. Mean Reward: 4.883349907055957. Std of Reward: 4.9962071469041645.
Step: 74000. Mean Reward: 5.0541429776247195. Std of Reward: 5.095435204606186.
Step: 75000. Mean Reward: 4.817925894644336. Std of Reward: 4.7088146222939.
Step: 76000. Mean Reward: 4.350448950712098. Std of Reward: 4.599494494658919.
Step: 77000. Mean Reward: 4.6857441705202785. Std of R

INFO:tensorflow:Restoring parameters from ./models/ppo\model-150001.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-150001.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-150001.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
